In [ ]:
%pip install numpy

In [14]:
import numpy as np

Scaled Dot product Attention

In [ ]:
def ScaledDotProductAttetnion(Q,K,V):
    scaled_scores = (np.matmul(Q,K.T))/np.sqrt(Q.shape[-1])
    exp_scores = np.exp(scaled_scores - np.max(scaled_scores, axis=-1,keepdims=True)) #subtract the max for numerical stability(Prevents np.exp() from overflowing when numbers are large)
    attention_weights = exp_scores / np.sum(exp_scores, axis=-1 ,keepdims=True)
    attention_score = np.matmul(attention_weights ,V)
    return attention_score



In [27]:
Q = np.array([[2,3,4],
               [5,5,6]])

K = np.array([[2,35,5],
               [5,5,4]])

V = np.array([[2,3,2],
               [5,4,5]])
attn_score = ScaledDotProductAttetnion(Q,K,V)
print(attn_score)

[[2. 3. 2.]
 [2. 3. 2.]]


In [36]:

arr = np.array([[[1, 2, 3],
                 [4, 5, 6]],

                [[7, 8, 9],
                 [10, 11, 12]]])
print(arr.shape)  
print(arr.transpose(0,2,1))

(2, 2, 3)
[[[ 1  4]
  [ 2  5]
  [ 3  6]]

 [[ 7 10]
  [ 8 11]
  [ 9 12]]]


Tensor Surgery(Splitting Heads ,reshaping , transposing , Combining heads )

In [48]:
def split_heads(tensor , batch_size , heads):
    dimension_head = int(tensor.shape[-1]/ heads) #shape[-1] takes the dimension of tensor 
    sequence_length = tensor.shape[1]
    x = tensor.reshape(batch_size , sequence_length , heads , dimension_head)
    final_tensor = x.transpose(0 , 2 , 1 , 3)
    return final_tensor


In [50]:
B , L , D = 32 , 10 , 512
H=8
example_tensor = np.zeros(( B , H , D )) # np.zeros mocks data
reshaped_tensor= split_heads(example_tensor,B,H )
print(f"Input Shape :{example_tensor.shape}")
print(f"Output  Shape :{reshaped_tensor.shape}")


Input Shape :(32, 8, 512)
Output  Shape :(32, 8, 8, 64)


In [1]:
def combine_heads(tensor ):
    batch_size = tensor.shape[0]
    heads = tensor.shape[1]
    sequence_length = tensor.shape[2]
    dimension_head = tensor.shape[3]
    x = tensor.transpose(0 , 2 , 1 , 3)
    combined_tensor = x.reshape(batch_size , sequence_length , heads * dimension_head)
    return combined_tensor

Sub-layers components

In [ ]:
def residual_connection( x , sublayer_output):
    return x + sublayer_output

def layer_normalization (residual_output ,gamma , beta , epsilon = 1e-5 ): #here gamma and beta are learnable parameters (weights and bias)
    mean = np.mean( residual_output , axis = -1 , keepdims = True)
    st_deviation= np.std( residual_output , axis = -1 , keepdims = True)
    normalized_output = ((residual_output -mean)/(st_deviation+epsilon) )*gamma + beta
    return normalized_output


The Feed-Forward Network(FFN)(In tranformer paper , FFN contains two linear layer + RELU activation function)

In [ ]:
def feed_forward(input_tensor , W1 , B1 , W2 , B2):
    linear_transform1 = np.matmul(input_tensor , W1) + B1
    activation_output = np.maximum (0 , linear_transform1) #ReLU activation
    linear_transform2 = np.matmul(activation_output , W2) + B2
    return linear_transform2


Positional encoding matrix

In [ ]:
def positional_encoding(max_sequence_length , dmodel):
    positional_encoding_matrix = np.zeros((max_sequence_length , dmodel))
    for pos in range(max_sequence_length):
        